# 5 Modeling Distributions

模型分布分为:

- 经验分布(Empirical Distribution):根据观察到的数据
- 理论分布(Theoretical Distribution):基于数学公式

这章主要介绍几种常见的理论分布, 我们可以通过这些分布公式计算概率, 从而可以构建PMF(Probability Mass Functions)(离散数据), CDF(Cumulative Distribution Function)(连续数据)

- 离散分布(Discrete Distribution)
  - 二项式分布(Binomial Distribution):用于模拟一系列成功或失败的次数, 例如击中目标的次数
  - 泊松分布(Poisson Distribution): 用于模拟在固定时间间隔内发生的事件次数, 例如一场比赛中的进球
- 连续分布(Continuous Distribution)
  - 指数分布(Exponential Distribution): 描述事件发生之间的时间间隔, 例如两次进球之间的时间
  - 正态分布(Normal Distribution): 也称高斯分布, 描述呈钟形曲线的数据, 例如婴儿的出生体重
  - 对数正态分布(Lognormal Distribution): 描述其对数值遵循正态分布的数据, 通常用于模拟像成人体重这样有偏斜的连续变量

In [ ]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

## 5.1 The Binomial Distribution

这里采用飞碟射击运动举例. 击中或击不中一个飞碟恰好是一个典型的二项式.


In [ ]:
import numpy as np

In [ ]:
def flip(n, p):
    """
    模拟二项试验（Binomial trials）中的随机结果，
    即生成一个由“成功”和“失败”组成的随机序列
    """
    choices = [1, 0]
    probs = [p, 1 - p]
    return np.random.choice(choices, size=n, p=probs)


def simulate_round(n, p):
    """
    模拟一轮射击, 返回击中次数
    """
    seq = flip(n, p)
    return seq.sum()

这是一个模拟 25 个靶标的回合示例，其中每个靶标的命中概率为 90%。

In [ ]:
np.random.seed(1)
flip(25, 0.9)

如果我们生成更长的序列并计算结果的 `Pmf` ，就能确认 1 和 0 的比例是正确的，至少大致如此。

In [ ]:
from empiricaldist import Pmf

seq = flip(1000, 0.9)
pmf = Pmf.from_seq(seq)
pmf

在一场大型比赛中，假设 200 名参赛者每人射击 5 轮，所有参赛者命中目标的概率相同，均为 `p=0.9` 。我们可以通过调用 `simulate_round` 1000 次来模拟这样的比赛。

In [ ]:
n = 25
p = 0.9
results_sim = [simulate_round(n, p) for _ in range(1000)]

平均得分接近 `22.5` ，这是 `n` 和 `p` 的乘积。

In [ ]:
np.mean(results_sim), n * p

以下是结果分布的情况。

In [ ]:
from empiricaldist import Pmf
from thinkstats import decorate

pmf_sim = Pmf.from_seq(results_sim, name="simulation results")

pmf_sim.bar()
decorate(xlabel="Hits", ylabel="PMF")

峰值接近均值，且分布向左偏斜。

我们本可以预测这一分布，而无需进行模拟。从数学上讲，这些结果的分布遵循二项分布，其概率质量函数易于计算。

二项式分布（二项分布）的概率质量函数公式为：

$$P(X = k) = C_n^k p^k (1-p)^{n-k}$$

或写作：

$$P(X = k) = \binom{n}{k} p^k q^{n-k}$$

其中：
- $P(X = k)$ 表示在 $n$ 次独立重复试验中，事件恰好发生 $k$ 次的概率
- $n$ 为试验总次数（$n \geq 1$ 的整数）
- $k$ 为成功次数（$k = 0, 1, 2, \ldots, n$）
- $p$ 为单次试验中事件发生的概率（$0 \leq p \leq 1$）
- $q = 1 - p$ 为单次试验中事件不发生的概率
- $C_n^k$ 或 $\binom{n}{k}$ 为组合数，表示从 $n$ 次试验中选出 $k$ 次成功的组合方式数，计算公式为 $\frac{n!}{k!(n-k)!}$

**公式含义**：该公式描述了在 $n$ 次独立伯努利试验中，成功次数为 $k$ 的概率分布。二项分布的期望值 $E(X) = np$，方差 $D(X) = np(1-p)$。

**典型应用场景**：抛硬币正面朝上的次数、产品合格率检验、投篮命中次数等"成功/失败"类型的重复独立试验问题。

---

**与泊松分布的关系**：当 $n$ 很大而 $p$ 很小时（通常 $n \geq 20$，$p \leq 0.05$），二项分布可用泊松分布近似，其中 $\lambda = np$。这就是泊松定理，也是为什么泊松分布常用于描述稀有事件的原因。

In [ ]:
from scipy.special import comb


def binomial_pmf(k, n, p):
    """
    计算二项分布的概率质量函数
    """
    return comb(n, k) * (p**k) * ((1 - p) ** (n - k))

`binomial_pmf` 函数在给定 `p` 的条件下，计算 `n` 次尝试中获得 `k` 次命中的概率。若我们使用一系列 k 值调用此函数，便能生成一个表示结果分布的 `Pmf` 。

In [ ]:
ks = np.arange(16, n + 1)
ps = binomial_pmf(ks, n, p)
pmf_binom = Pmf(ps, ks, name="binomial model")
ks, ps

In [ ]:
from thinkstats import two_bar_plots

two_bar_plots(pmf_sim, pmf_binom)
decorate(xlabel="Hits", ylabel="PMF")

两者结果相似，仅因模拟结果的随机波动存在细微差异。这种一致性并不令人意外，因为模拟和模型基于相同的假设——特别是每次尝试具有相同成功概率的前提。对模型更严格的检验在于其与实际数据的吻合程度。

从 2020 年夏季奥运会男子双向飞碟射击比赛的维基百科页面，我们可以提取一张显示资格赛结果的表格。

In [ ]:
filename = "Shooting_at_the_2020_Summer_Olympics_Mens_skeet"
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/" + filename)

In [ ]:
import pandas as pd

tables = pd.read_html(filename)
table = tables[6]
table.head()

每位参赛者对应表格中的一行，每轮比赛占据一列。我们将选取包含这些结果的列，并使用 NumPy 函数 `flatten` 将它们整合到单个数组中。

In [ ]:
columns = ["1", "2", "3", "4", "5"]
results = table[columns].values.flatten()
results

在 30 名参赛者中，我们获得了 150 轮比赛的结果，每轮射击 25 发，总共 3750 次尝试中命中了 3575 次。

In [ ]:
total_shots = n * len(results)
total_hits = results.sum()
n, total_shots, total_hits

因此总体成功率为 95.3%。

In [ ]:
p = total_hits / total_shots
p

现在让我们计算一个 Pmf ，它代表具有 n=25 的二项分布以及我们刚刚计算的 p 值。

In [ ]:
ps = binomial_pmf(ks, n, p)
pmf_binom = Pmf(ps, ks, name="binomial model")

我们可以将其与实际结果的 Pmf 进行比较。

In [ ]:
pmf_results = Pmf.from_seq(results, name="actual results")

two_bar_plots(pmf_results, pmf_binom)
decorate(xlabel="Hits", ylabel="PMF")

二项模型与数据分布高度吻合——尽管它做出了不切实际的假设，即所有竞争者都具备相同且恒定的能力。

## 5.2 The Poisson Distribution

另一个体现体育赛事结果遵循可预测模式的例子是冰球比赛中的进球数。

我们将从模拟一场 60 分钟的比赛开始，即 3600 秒，假设每场比赛平均总共进球 6 个，且在任何一秒内进球的概率 `p` 都相同。

In [ ]:
n = 3600
m = 6
p = m / 3600
p

现在我们可以使用以下函数来模拟 n 秒并返回进球总数。

In [ ]:
def simulate_goals(n, p):
    return flip(n, p).sum()

如果我们模拟多场比赛，可以确认每场比赛的平均进球数接近 6 个。

现在我们模拟1001场比赛, 每场比赛3600秒, 每秒进球 `6 / 3600`

In [ ]:
goals = [simulate_goals(n, p) for i in range(1001)]
np.mean(goals)

我们可以使用二项分布来模拟这些结果，但当比赛场次较多且单场进球概率较低时，泊松分布也能很好地模拟这些结果。泊松分布由一个通常用希腊字母λ表示的值来定义，该字母读作“lambda”，在代码中用变量`lam`表示（`lambda`不能作为合法变量名，因为它是 Python 关键字）。`lam`代表进球率，在本例中为每场比赛 6 个进球。

泊松分布的概率质量函数（PMF）公式为：

$$P(X = k) = \frac{\lambda^k e^{-\lambda}}{k!}$$

其中：
- $P(X = k)$ 表示随机变量 $X$ 取值为 $k$ 的概率
- $\lambda$ 是单位时间（或单位空间）内事件发生的平均次数（$\lambda > 0$）
- $k$ 是实际发生次数（$k = 0, 1, 2, \ldots$）
- $e$ 是自然常数（约等于 2.71828）
- $k!$ 是 $k$ 的阶乘

**公式含义**：该公式描述了在固定时间或空间内，事件发生 $k$ 次的概率，适用于描述**稀有事件**在大量独立试验中的发生情况。泊松分布的期望和方差都等于 $\lambda$。

**典型应用场景**：单位时间内接到的电话呼叫数、网站访问量、机器故障次数、放射性粒子衰变数等低概率、独立发生的事件计数问题。


In [ ]:
from scipy.special import factorial


def poisson_pmf(k, lam):
    """Compute the Poisson PMF.

    k (int or array-like): The number of occurrences
    lam (float): The rate parameter (λ) of the Poisson distribution

    returns: float or ndarray
    """
    return (lam**k) * np.exp(-lam) / factorial(k)

如果我们用一系列 `k` 值调用 `poisson_pmf` ，就能生成一个代表结果分布的 `Pmf` 。

In [ ]:
lam = 6
ks = np.arange(20)
ps = poisson_pmf(ks, lam)
pmf_poisson = Pmf(ps, ks, name="Poisson model")

并确认该分布的均值接近 6。

In [ ]:
pmf_poisson.normalize()
pmf_poisson.mean()

下图将模拟结果与具有相同均值的泊松分布进行了比较。

In [ ]:
pmf_sim = Pmf.from_seq(goals, name="simulation")

two_bar_plots(pmf_sim, pmf_poisson)
decorate(xlabel="Goals", ylabel="PMF")

除了因随机波动造成的微小差异外，这些分布情况基本相似。这并不令人意外，因为模拟和泊松模型都基于同一假设：在比赛的任何一秒内进球的概率是相同的。因此，更严格的检验是看模型与真实数据的拟合程度如何。



我从 HockeyReference 网站下载了 2023-2024 赛季国家冰球联盟（NHL）常规赛（不包括季后赛）的所有比赛结果。我提取了 60 分钟常规比赛时间内的进球信息，不包括加时赛或决胜射门环节。数据保存在一个 HDF 文件中，每场比赛对应一个键值，其中记录了每个进球发生的时间（以比赛开始后的秒数表示）。本章节的笔记本中提供了下载这些数据的详细说明。

In [ ]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/nhl_2023_2024.hdf")

In [ ]:
filename = "nhl_2023_2024.hdf"

with pd.HDFStore(filename, "r") as store:
    keys = store.keys()

len(keys), keys[0]

常规赛季期间共有 1312 场比赛。每个键包含比赛日期和主队的三字母缩写。我们可以使用 read_hdf 来查找键并获取进球得分的时间列表。

In [ ]:
times = pd.read_hdf(filename, key=keys[0])
times

赛季首场比赛中，共打入六球，首球出现在开赛 424 秒后，末球则在比赛仅剩 87 秒时的第 3513 秒攻入。

In [ ]:
3600 - times[5]

以下循环读取所有比赛的结果，统计每场比赛的进球数，并将结果存储在一个列表中。

In [ ]:
goals = []

for key in keys:
    times = pd.read_hdf(filename, key=key)
    n = len(times)
    goals.append(n)

每场比赛的平均进球数略高于 6 个。

In [ ]:
lam = np.mean(goals)
lam

我们可以使用 poisson_pmf 来创建一个 Pmf ，使其代表一个与数据具有相同均值的泊松分布。

In [ ]:
ps = poisson_pmf(ks, lam)
pmf_poisson = Pmf(ps, ks, name="Poisson model")

与数据的概率质量函数相比，其形态如下所示。

In [ ]:
pmf_goals = Pmf.from_seq(goals, name="goals scored")

two_bar_plots(pmf_goals, pmf_poisson)
decorate(xlabel="Goals", ylabel="PMF")